## Pre-compute and store line segmentations

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from ast import literal_eval

In [ ]:
# torch
import torch
import torchvision
# addons
from tqdm import tqdm

In [ ]:
#%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
relative_path = '../../'
# ensure that parent path is on the python path in order to have all packages available
import sys, os
parent_path = os.path.join(os.getcwd(), relative_path)
parent_path = os.path.realpath(parent_path)  # os.path.abspath(...)
sys.path.insert(0, parent_path)

In [ ]:
from lib.models.trained_model_loader import get_line_net_fcn
from lib.datasets.cunei_dataset_segments import CuneiformSegments, get_segment_meta
from lib.transliteration.sign_labels import get_label_list
from lib.utils.transform_utils import UnNormalize

from lib.detection.run_gen_line_detection import gen_line_detections

### Config Basics

In [ ]:
# toggle generation
save_line_detections = True

In [ ]:
# set line segmentation network
line_model_version = 'v002'

In [ ]:
# dataset config
collections = ['test', 'train', 'saa01',  'saa05', 'saa06', 'saa08', 'saa09', 'saa10', 'saa13', 'saa16'] 
#collections = ['saa01', 'saa05']

In [ ]:
data_layer_params = dict(batch_size=[128, 16],
                         img_channels=1,
                         gray_mean=[0.5],
                         gray_std=[1.0], 
                         num_classes = 2
                         )

### Config Data Augmentation

In [ ]:
num_classes = data_layer_params['num_classes']
num_c = data_layer_params['img_channels']
gray_mean = data_layer_params['gray_mean']
gray_std = data_layer_params['gray_std']

In [ ]:
re_transform = torchvision.transforms.Compose([
    UnNormalize(mean=gray_mean, std=gray_std),
    torchvision.transforms.ToPILImage(),
                                              ])
re_transform_rgb = torchvision.transforms.Compose([
    UnNormalize(mean=gray_mean * 3, std=gray_std * 3),
    torchvision.transforms.ToPILImage(),
                                              ])

### Load Model

In [ ]:
#use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model_fcn = get_line_net_fcn(line_model_version, device, num_classes=num_classes, num_c=num_c)
print(model_fcn)

### Run experiment

In [ ]:
for saa_version in collections:
    print('collection: <><>{}<><>'.format(saa_version))
    
    ### Get collection dataset
    dataset = CuneiformSegments(collections=[saa_version], relative_path=relative_path, 
                                only_annotated=False, only_assigned=True, preload_segments=False)
    
    # filter collection dataset - OPTIONAL
    didx_list = range(len(dataset))
    
    ### Generate line detections
    gen_line_detections(didx_list, dataset, saa_version, relative_path,
                        line_model_version, model_fcn, re_transform, device,
                        save_line_detections)     